In [11]:
from bs4 import BeautifulSoup
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time
import datetime
import pandas as pd

In [ ]:
!which chromedriver

In [18]:
executable_path = {'executable_path': ChromeDriverManager().install()}
# browser = Browser('chrome', **executable_path)
browser = Browser('chrome', **executable_path, headless=False)
# browser = Browser('chrome')

[WDM] - Current google-chrome version is 83.0.4103
[WDM] - Get LATEST driver version for 83.0.4103
[WDM] - Driver [/Users/abby/.wdm/drivers/chromedriver/mac64/83.0.4103.39/chromedriver] found in cache


In [19]:
base_url = 'https://www.walmart.com'
search_url = '/search/?query=room%20air%20purifier'
browser.visit(base_url + search_url)

In [20]:
html = browser.html
soup = BeautifulSoup(html, 'html.parser')

In [21]:
products = soup.find_all('div', class_='search-result-gridview-item')

In [22]:
# Create empty lists for scraped data to be stored in.
productURL_list = []
productImage_list = []
productTitle_list = []
starReview_list = []
reviewAmount_list = []
currentPrice_list = []
reviewURL_list = []


today = datetime.datetime.now()

for product in products:

    # Get the link to the product page.
    link = product.find('a')
    href = link['href']
    product_url = base_url + href
    productURL_list.append(product_url)

    # Get the link to the product image.
    img = product.find('img')['src']
    productImage_list.append(img)

    # Get the product title.
    product_title = product.find('img')['alt']
    productTitle_list.append(product_title)

    # Try to get the number of reviews, there may not be any.
    try:
        review_amount = product.find('span', class_='seo-review-count visuallyhidden').text
        reviewAmount_list.append(review_amount)
        
    except:
        review_amount = 0
        reviewAmount_list.append(review_amount)

    # In the case where there are at least one review...
    if int(review_amount) > 0:
        # Get the average number of stars
        stars_review = product.find('span', class_='visuallyhidden seo-avg-rating').text   
        starReview_list.append(stars_review)
        
        # Get the URL to the reviews section for that product.
        review_url = product.find('div', class_='stars').find('a')['href']
        reviewURL_list.append(review_url)

    # Otherwise, use defalt 0 or NaN values for these entries.
    else:
        stars_review = 0
        starReview_list.append(stars_review)
        
        review_url = "NaN"
        reviewURL_list.append(review_url)
        
    # Get the price of the product.
    price = product.find('span', class_='price-main')
    current_price = price.find('span', class_='visuallyhidden').text
    currentPrice_list.append(current_price)
    
# Close the browser window
# browser.quit()
    
# Create a dictionary with the lists of the scrapped data.
data = {
    "Title": productTitle_list,
    "URL": productURL_list,
    "Image": productImage_list,
    "NumberOfStars": starReview_list,
    "NumberofReviews": reviewAmount_list,
    "ReviewsURL": reviewURL_list,
    "Price": currentPrice_list
       }

# Create a Pandas DataFrame with that dictionary
product_df = pd.DataFrame.from_dict(data)
product_df.head()

,Title,URL,Image,NumberOfStars,NumberofReviews,ReviewsURL,Price
0,"Filtrete by 3M Room Air Purifier, Console, 110...",https://www.walmart.com/ip/Filtrete-by-3M-Room...,https://i5.walmartimages.com/asr/eae28c2f-5dbe...,4.5,184,/ip/Filtrete-by-3M-Room-Air-Purifier-Console-1...,$49.94
1,Holmes Personal Space 2-Speed Air Purifier wit...,https://www.walmart.com/ip/Holmes-Personal-Spa...,https://i5.walmartimages.com/asr/ca7e3b65-d3c8...,4.3,510,/ip/Holmes-Personal-Space-2-Speed-Air-Purifier...,$14.98
2,Filtrete F2 Bacteria & Virus True HEPA Room Ai...,https://www.walmart.com/ip/Filtrete-F2-Bacteri...,https://i5.walmartimages.com/asr/dea94bac-be28...,4.8,55,/ip/Filtrete-F2-Bacteria-Virus-True-HEPA-Room-...,$17.97
3,High-Efficiency HEPA-Type Desktop Air Purifier...,https://www.walmart.com/ip/High-Efficiency-HEP...,https://i5.walmartimages.com/asr/5ba9740e-a3f1...,4.4,136,/ip/High-Efficiency-HEPA-Type-Desktop-Air-Puri...,$49.69
4,TENDOMI WiFi Smart Air Purifier with True HEPA...,https://www.walmart.com/ip/TENDOMI-WiFi-Smart-...,https://i5.walmartimages.com/asr/a4b27232-85c0...,4.5,20,/ip/TENDOMI-WiFi-Smart-Air-Purifier-True-HEPA-...,$65.99


In [32]:
# print(base_url+product_df["ReviewsURL"][0])

browser.visit(base_url+product_df["ReviewsURL"][0])
html = browser.html
soup = BeautifulSoup(html, 'html.parser')
allReviewsURL = soup.find('a', class_="button ReviewBtn-container ReviewsHeader-seeAll button--primary")['href']
browser.visit(base_url+allReviewsURL)
soup = BeautifulSoup(html, 'html.parser')

In [36]:
reviews = soup.find('div', class_='review-highlight')
positive = reviews.find('div', class_='font-bold highlight-title').text
stars = reviews.find('div', class_='highlight-review-stars').find('span', class_='average-rating').text
body = reviews.find('div', class_='collapsable-content-container').text
print(positive)
print(stars)
print(body)

Most helpful positive review
(5.0)
5 stars after looking online for the perfcet ir purifier i found these one budget friendly the filter is huge 3m brand the best quality they have them at the store or online  for $17 its powerful in the full speed no digital buttons just the knob and the filter change reminder light works perfect i recommend keep in  second speed to extend the filter life


In [ ]:
soup.execute_script("window.scrollTo(0, document.body.scrollHeight);")